Data can be download from [kraggle](https://www.kaggle.com/c/home-credit-default-risk/data) or directly [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Impl%C3%A9menter+un+mod%C3%A8le+de+scoring/Projet+Mise+en+prod+-+home-credit-default-risk.zip). The projet concern a scoring score about cease of payment. Interactif dashboards are also present to understand the parameters behind the algorithm and the reason behind a refusal. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from bokeh.plotting import output_notebook, figure, show
from bokeh.models import ColumnDataSource, Div, Select, Button, ColorBar, CustomJS
from bokeh.layouts import row, column, layout
from bokeh.transform import cumsum, linear_cmap
from bokeh.palettes import Blues8

output_notebook()
pd.set_option('display.max_columns', None)

In [ ]:
def distance_IQ(dt, q1=0.25, q3=0.75, return_flag = False):
    """
    Compute interquantile value and return number
    of elements 
        * Above Q3 + 1.5*IQ
        * Bellow Q1 - 1*5 IQ
    Arguments:
        _dt: dataFrame (pandas)
        _q1: first quantile (default 0.25)
        _q3: third quantile (default 0.75)
    """
    q1 = dt.quantile(q1)
    q3 = dt.quantile(q3)
    IQ = q3-q1
    count = dt[dt < q1 - 1.5* IQ ].count() 
    print("Element {0} bellow Q1 - 1.5 * IQ ({1:.2f})".format(count, q1 - 1.5* IQ))
    count = dt[dt > q3 + 1.5* IQ ].count() 
    print("Element {0} above Q3 + 1.5 * IQ ({1:.2f})".format(count ,q3 + 1.5 * IQ))
    if return_flag:
        return q1 - 1.5* IQ, q3 + 1.5* IQ

In [ ]:
def clean_IQ(dt, lab, q1=0.25, q3=0.75):
    borne_inf, borne_sup = distance_IQ(dt[lab], q1=0.25, q3=0.75, return_flag = True)
    if borne_inf - borne_sup == 0:
        print("distance IQ = 0")
        return dt
    if borne_inf is not None:
        dt = dt[dt[lab] > borne_inf]
    if borne_sup is not None:
        dt = dt[dt[lab] < borne_sup]
    return dt

# Load data



In [ ]:
df_test = pd.read_csv('application_test.csv')
df_train = pd.read_csv('application_train.csv')
df_bureau = pd.read_csv('bureau.csv')
df_bureau_balance = pd.read_csv('bureau_balance.csv')
df_credit_balance = pd.read_csv('credit_card_balance.csv')
df_home_credit = pd.read_csv("HomeCredit_columns_description.csv", encoding = "ISO-8859-1")
df_installments_payments = pd.read_csv("installments_payments.csv")
df_POS_CASH_balance = pd.read_csv("POS_CASH_balance.csv")
df_previous_application = pd.read_csv("previous_application.csv")
df_sample_submission = pd.read_csv("sample_submission.csv")

### Test

In [ ]:
df_test.head(3)


### Train

In [ ]:
df_train.head(3)
#This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
#Static data for all applications. One row represents one loan in our data sample.

In [ ]:
data_train = df_train[["SK_ID_CURR", "TARGET", "NAME_CONTRACT_TYPE", 
                      "CODE_GENDER", "FLAG_OWN_CAR", "FLAG_OWN_REALTY",
                      "CNT_CHILDREN", "AMT_INCOME_TOTAL", "AMT_CREDIT", 
                      "AMT_ANNUITY", "AMT_GOODS_PRICE", "NAME_INCOME_TYPE",
                       "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE",
                      "DAYS_BIRTH", "DAYS_EMPLOYED", "OWN_CAR_AGE",
                      "OCCUPATION_TYPE", "CNT_FAM_MEMBERS"]]

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "LANDAREA_MODE", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "TARGET", "Description"].values)

In [ ]:
data_train.hist("TARGET", density=1)

#### OCCUPATION_TYPE

In [ ]:
data_train["OCCUPATION_TYPE"].value_counts(normalize = True).plot.bar()

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "OCCUPATION_TYPE", "Description"].values)

#### DAYS_EMPLOYED

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "DAYS_EMPLOYED", "Description"].values)

In [ ]:
data_train[data_train["DAYS_EMPLOYED"] < 50000].hist("DAYS_EMPLOYED")

In [ ]:
data_train[(data_train["DAYS_EMPLOYED"] >0) & (data_train["DAYS_EMPLOYED"] < 500000)].hist("DAYS_EMPLOYED")

**300 000 jours correspond à plus de 87 ans de travail**. On supprime les valeurs positives

In [ ]:
#distance_IQ(data_train["DAYS_EMPLOYED"])
data_train = clean_IQ(data_train, "DAYS_EMPLOYED")

In [ ]:
data_train.hist("DAYS_EMPLOYED")

#### NAME_CONTRACT_TYPE

In [ ]:
data_train["NAME_CONTRACT_TYPE"].value_counts()

#### CODE_GENDER

In [ ]:
data_train["CODE_GENDER"].value_counts()

On retire le genre indéfini pour éviter de futur biais.

In [ ]:
data_train = data_train[df_train["CODE_GENDER"] != "XNA"]

#### AMT_INCOME_TOTAL

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_INCOME_TOTAL", "Description"].values)

In [ ]:
data_train.hist("AMT_INCOME_TOTAL", bins = 50)

In [ ]:
data_train["AMT_INCOME_TOTAL"].describe()

3/4 des individus ont moins de 200 000 euros de revenu. Et un individu gagne plus de 10 millions de dolars. 

In [ ]:
data_train = clean_IQ(data_train, "AMT_INCOME_TOTAL")
data_train.hist("AMT_INCOME_TOTAL", bins = 20)

#### AMT_CREDIT

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT", "Description"].values)

In [ ]:
data_train.hist("AMT_CREDIT", bins= 50)

In [ ]:
data_train = clean_IQ(data_train, "AMT_CREDIT")
data_train.hist("AMT_CREDIT", bins = 50)

#### AMT_ANNUITY

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_ANNUITY", "Description"].values)

In [ ]:
data_train.hist("AMT_ANNUITY", bins= 50)

#### AMT_GOODS_PRICE

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_GOODS_PRICE", "Description"].values)

In [ ]:
data_train.hist("AMT_GOODS_PRICE", bins= 50)

#### AMT_REQ_CREDIT_BUREAU_X

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_REQ_CREDIT_BUREAU_HOUR", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_REQ_CREDIT_BUREAU_DAY", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "FLAG_OWN_REALTY", "Description"].values)

#### good data

In [ ]:
data_train.head()

In [ ]:
data_train.shape

In [ ]:
data_train =pd.get_dummies(data_train, columns=["NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "FLAG_OWN_CAR", 
                                    "FLAG_OWN_REALTY", "NAME_FAMILY_STATUS", "NAME_CONTRACT_TYPE", 
                                    "CODE_GENDER", 'NAME_HOUSING_TYPE'], drop_first=True)

In [ ]:
data_train.shape

In [ ]:
data_train.head()

## Bureau

In [ ]:
df_bureau.head(3)
#All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
#For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

In [ ]:
good_bureau = df_bureau[["SK_ID_CURR", "SK_ID_BUREAU", "CREDIT_ACTIVE", "CREDIT_CURRENCY", 
                         "DAYS_CREDIT", "CREDIT_DAY_OVERDUE", "DAYS_CREDIT_ENDDATE", 
                         "DAYS_ENDDATE_FACT", "AMT_CREDIT_MAX_OVERDUE", "CNT_CREDIT_PROLONG", 
                         "AMT_CREDIT_SUM", "AMT_CREDIT_SUM_DEBT"]]

In [ ]:
good_bureau.shape

In [ ]:
good_bureau["CREDIT_ACTIVE"].value_counts(normalize = True).plot.bar()

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "CREDIT_CURRENCY", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "CREDIT_DAY_OVERDUE", "Description"].values)

In [ ]:
good_bureau["CREDIT_DAY_OVERDUE"].describe()

In [ ]:
good_bureau.hist("CREDIT_DAY_OVERDUE")

In [ ]:
good_bureau = clean_IQ(good_bureau, "CREDIT_DAY_OVERDUE")

### Somme des crédits et durée

In [ ]:
good_bureau.hist("DAYS_CREDIT_ENDDATE", bins = 50)

In [ ]:
good_bureau["DAYS_CREDIT_ENDDATE"].describe()

In [ ]:
good_bureau = clean_IQ(good_bureau, "DAYS_CREDIT_ENDDATE")

In [ ]:
good_bureau.hist("DAYS_CREDIT_ENDDATE", bins = 50)

In [ ]:
good_bureau.hist("AMT_CREDIT_SUM", bins = 50)

In [ ]:
good_bureau = clean_IQ(good_bureau, "AMT_CREDIT_SUM")

In [ ]:
good_bureau.hist("AMT_CREDIT_SUM", bins = 50)

In [ ]:
y = good_bureau["AMT_CREDIT_SUM"].to_numpy()
x = good_bureau["DAYS_CREDIT_ENDDATE"].to_numpy()
plt.scatter(x, y, alpha = 0.2)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "CREDIT_ACTIVE", "Description"].values)

In [ ]:
good_bureau["CREDIT_ACTIVE"].value_counts()
#keep only row whith recent update

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "CREDIT_CURRENCY", "Description"].values)

In [ ]:
good_bureau["CREDIT_CURRENCY"].value_counts()
#keep only row whith recent update

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "DAYS_CREDIT", "Description"].values)

In [ ]:
df_bureau.hist("DAYS_CREDIT", bins = 50)
#keep only row whith recent update

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "DAYS_CREDIT_ENDDATE", "Description"].values)

In [ ]:
good_bureau.hist("DAYS_CREDIT_ENDDATE", bins = 50)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "DAYS_ENDDATE_FACT", "Description"].values)

In [ ]:
good_bureau.hist("DAYS_ENDDATE_FACT", bins = 50)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_SUM_DEBT", "Description"].values)

In [ ]:
good_bureau.hist("AMT_CREDIT_SUM_DEBT", bins = 50)

In [ ]:
good_bureau.shape

In [ ]:
good_bureau.head()

In [ ]:
good_bureau = pd.get_dummies(good_bureau, columns=["CREDIT_ACTIVE", "CREDIT_CURRENCY"])

In [ ]:
good_bureau.head()

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_SUM_DEBT", "Description"].values)

### Sous tableau informations des crédits en cours

In [ ]:
bureau_credit_active = good_bureau[good_bureau["CREDIT_ACTIVE_Active"] == 1]
bureau_credit_active = bureau_credit_active[["SK_ID_CURR", "AMT_CREDIT_MAX_OVERDUE", "CREDIT_ACTIVE_Active",
                                             "AMT_CREDIT_SUM", ]].groupby(["SK_ID_CURR"]).sum()
#rename columns
bureau_credit_active[
    "AMT_CREDIT_MAX_OVERDUE"
] = bureau_credit_active["AMT_CREDIT_MAX_OVERDUE"]/bureau_credit_active["AMT_CREDIT_SUM"]
bureau_credit_active = bureau_credit_active.rename(columns={"AMT_CREDIT_MAX_OVERDUE": "proportion_OVERDUE_active",
                                                            "CREDIT_ACTIVE_Active" : "CREDIT_Active",
                                                            "AMT_CREDIT_SUM": "AMT_CREDIT_SUM_active"})
bureau_credit_active = bureau_credit_active.reset_index()

### Sous tableau informations des crédits fermés

In [ ]:
bureau_credit_closed = good_bureau[good_bureau["CREDIT_ACTIVE_Closed"] == 1]
bureau_credit_closed = bureau_credit_closed[["SK_ID_CURR", "AMT_CREDIT_MAX_OVERDUE", "CREDIT_ACTIVE_Closed",
                                             "AMT_CREDIT_SUM"]].groupby(["SK_ID_CURR"]).sum()
id_curr = bureau_credit_closed.index.to_numpy()
#rename columns
bureau_credit_closed[
    "AMT_CREDIT_MAX_OVERDUE"
] = bureau_credit_closed["AMT_CREDIT_MAX_OVERDUE"]/bureau_credit_closed["AMT_CREDIT_SUM"]
bureau_credit_closed = bureau_credit_closed.rename(columns={"AMT_CREDIT_MAX_OVERDUE": "proportion_OVERDUE_closed",
                                                            "CREDIT_ACTIVE_Closed" : "CREDIT_closed",
                                                            "AMT_CREDIT_SUM": "AMT_CREDIT_SUM_closed"})
bureau_credit_closed = bureau_credit_closed.reset_index()

### Sous tableau informations des crédits vendus

In [ ]:
bureau_credit_sold = good_bureau[good_bureau["CREDIT_ACTIVE_Sold"] == 1]
bureau_credit_sold = bureau_credit_sold[["SK_ID_CURR", "AMT_CREDIT_MAX_OVERDUE", "CREDIT_ACTIVE_Sold",
                                             "AMT_CREDIT_SUM"]].groupby(["SK_ID_CURR"]).sum()
#rename columns
bureau_credit_sold[
    "AMT_CREDIT_MAX_OVERDUE"
] = bureau_credit_sold["AMT_CREDIT_MAX_OVERDUE"]/bureau_credit_sold["AMT_CREDIT_SUM"]
bureau_credit_sold = bureau_credit_sold.rename(columns={"AMT_CREDIT_MAX_OVERDUE": "proportion_OVERDUE_sold",
                                                        "CREDIT_ACTIVE_Sold" : "CREDIT_sold",
                                                        "AMT_CREDIT_SUM": "AMT_CREDIT_SUM_sold"})
bureau_credit_sold = bureau_credit_sold.reset_index()

### Sous tableau informations des mauvais crédits

In [ ]:
bureau_credit_bad_debt = good_bureau[good_bureau["CREDIT_ACTIVE_Bad debt"] == 1]
bureau_credit_bad_debt = bureau_credit_bad_debt[["SK_ID_CURR", "AMT_CREDIT_MAX_OVERDUE", "CREDIT_ACTIVE_Bad debt",
                                             "AMT_CREDIT_SUM"]].groupby(["SK_ID_CURR"]).sum()
#rename columns
bureau_credit_bad_debt[
    "AMT_CREDIT_MAX_OVERDUE"
    ] = bureau_credit_bad_debt["AMT_CREDIT_MAX_OVERDUE"]/bureau_credit_bad_debt["AMT_CREDIT_SUM"]
bureau_credit_bad_debt = bureau_credit_bad_debt.rename(columns={"AMT_CREDIT_MAX_OVERDUE": "proportion_OVERDUE_bad",
                                                                "CREDIT_ACTIVE_Bad debt" : "CREDIT_bad",
                                                        "AMT_CREDIT_SUM": "AMT_CREDIT_SUM_bad"})
bureau_credit_bad_debt = bureau_credit_bad_debt.reset_index()

### Merge tableau

In [ ]:
data_train = data_train.merge(bureau_credit_active, on='SK_ID_CURR', how='left')
data_train = data_train.merge(bureau_credit_sold, on='SK_ID_CURR', how='left')
data_train = data_train.merge(bureau_credit_closed, on='SK_ID_CURR', how='left')
data_train = data_train.merge(bureau_credit_bad_debt, on='SK_ID_CURR', how='left')

In [ ]:
for col in data_train.columns[36:]:
    data_train[col] = data_train[col].fillna(0)

Proportion des crédits qui sont arrivés à terme, mauvais et vendu

In [ ]:
data_train.CREDIT_sold = data_train.CREDIT_sold / (data_train.CREDIT_sold +  data_train.CREDIT_closed + data_train.CREDIT_bad)
data_train.CREDIT_closed = data_train.CREDIT_closed / (data_train.CREDIT_sold +  data_train.CREDIT_closed + data_train.CREDIT_bad)
data_train.CREDIT_bad = data_train.CREDIT_bad / (data_train.CREDIT_sold +  data_train.CREDIT_closed + data_train.CREDIT_bad)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_MAX_OVERDUE", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_SUM", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_SUM_DEBT", "Description"].values)

In [ ]:
bureau_credit_detail = good_bureau[["SK_ID_CURR", "CREDIT_DAY_OVERDUE", "DAYS_CREDIT_ENDDATE",
                "AMT_CREDIT_MAX_OVERDUE"]]

## Bureau balance

In [ ]:
df_bureau_balance.head(3)
#Monthly balances of previous credits in Credit Bureau.
#This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has 
#(#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "MONTHS_BALANCE", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "STATUS", "Description"].values)

## Crédit balance

In [ ]:
df_credit_balance.head(3)
#Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
#This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

In [ ]:
df_credit_balance.hist("AMT_BALANCE", bins = 50)

In [ ]:
distance_IQ(df_credit_balance["AMT_BALANCE"])

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_LIMIT_ACTUAL", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_DRAWINGS_ATM_CURRENT", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "NAME_CONTRACT_STATUS", "Description"].values)

In [ ]:
df_credit_balance["NAME_CONTRACT_STATUS"].value_counts(normalize = True).plot.bar()

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_PAYMENT_TOTAL_CURRENT", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_TOTAL_RECEIVABLE", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "MONTHS_BALANCE", "Description"].values)

In [ ]:
df_credit_balance.hist("MONTHS_BALANCE", bins = 50)

Data are not update every month. Most are older than 1 year.

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_BALANCE", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_LIMIT_ACTUAL", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_DRAWINGS_CURRENT", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_DRAWINGS_ATM_CURRENT", "Description"].values)

In [ ]:
df_credit_balance[df_credit_balance["AMT_BALANCE"] < 600000].hist("AMT_BALANCE", bins = 50)
#df_credit_balance.hist("AMT_BALANCE", bins = 50)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "CNT_INSTALMENT_MATURE_CUM", "Description"].values)

In [ ]:
df_credit_balance.hist("AMT_BALANCE", bins = 50)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_CREDIT_LIMIT_ACTUAL", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_DRAWINGS_ATM_CURRENT", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_DRAWINGS_CURRENT", "Description"].values)

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_DRAWINGS_OTHER_CURRENT", "Description"].values)

In [ ]:
df_credit_balance.describe()

In [ ]:
df_credit_balance = clean_IQ(df_credit_balance, "AMT_BALANCE")
df_credit_balance.hist("AMT_BALANCE", bins = 50)

In [ ]:
tmp = df_credit_balance.groupby(['SK_ID_CURR','NAME_CONTRACT_STATUS']).sum().reset_index()[
    ["SK_ID_CURR", "NAME_CONTRACT_STATUS", "AMT_BALANCE", "AMT_CREDIT_LIMIT_ACTUAL"]
    ]
tmp2 = pd.DataFrame({"Ratio_Credit_limit" : tmp["AMT_BALANCE"]/tmp["AMT_CREDIT_LIMIT_ACTUAL"]})
good_balance = pd.concat([tmp[['SK_ID_CURR']], tmp2], axis=1)

In [ ]:
good_balance

In [ ]:
good_balance.replace([np.inf, -np.inf], np.nan, inplace=True)
good_balance.describe()

In [ ]:
print(df_home_credit.loc[df_home_credit["Row"] == "AMT_BALANCE", "Description"].values)

In [ ]:
df_credit_balance.head(15)
#Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
#This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

## Home crédit

In [ ]:
df_home_credit.head(10)
#explain label

## Installments payments

In [ ]:
df_installments_payments.head(3)
#Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
#There is a) one row for every payment that was made plus b) one row each for missed payment.
#One row is equivalent to one payment of one installment OR one installment corresponding to one payment 
#of one previous Home Credit credit related to loans in our sample.

In [ ]:
df_installments_payments.shape

## POS CASH balance

In [ ]:
df_POS_CASH_balance.head(3)
#Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
#This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans)
#related to loans in our sample – i.e. the table has 
#(#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

In [ ]:
df_POS_CASH_balance.shape

In [ ]:
df_home_credit[142:150]

## previous_application

In [ ]:
df_previous_application.head(3)
#All previous applications for Home Credit loans of clients who have loans in our sample.
#There is one row for each previous application related to loans in our data sample.

In [ ]:
df_previous_application.shape

## sample submission

In [ ]:
df_sample_submission.head(3)

In [ ]:
df_previous_application.shape

# 2. Interactive bar chart with text display block

The first chart will display the sex (female or male) of Titanic survivors depending on the lifeboats which they boarded. For this purpose I will be using an interactive bar chart which will display the number of people when hovering over the bars. To make this chart more interactive, I will add an HTML content block that will display additional text information about the lifeboat survivors when one of the bars is selected.

I start by creating a new DataFrame `df1` with the columns `Lifeboat` and `Sex`. Don't forget to set is as a copy to avoid `SettingWithCopyWarning` further down the code.

In [ ]:
df1 = full[['Lifeboat', 'Sex']].copy()
df1.head()

I filter out the entries that contain `NaN` or `?` in the column `Lifeboat` and remove the unnecessary symbols. Then I create dummy variables from the column `Sex` in order to count how many females/males were present in each lifeboat. The prefix and the separator of dummy variables are made empty to avoid long column names.

In [ ]:
df1 = df1[(df1.Lifeboat.notna()) & (df1.Lifeboat != '?')]

df1.loc[df1.Lifeboat == '14?', 'Lifeboat'] = '14'
df1.loc[df1.Lifeboat == '15?', 'Lifeboat'] = '15'
df1.loc[df1.Lifeboat == 'A[64]', 'Lifeboat'] = 'A'

df1 = pd.get_dummies(df1, columns=['Sex'], prefix='', prefix_sep='')

df1.head()

Then I group the rows according to the values in `Lifeboat` and sum up the number of females/males.

In [ ]:
df1 = df1.groupby('Lifeboat', as_index=False).sum()

df1.head()

I thought that it would be interesting to arrange lifeboats according to the order in which they were launched (taken from [Wikipedia](https://en.wikipedia.org/wiki/Lifeboats_of_the_RMS_Titanic)).

In [ ]:
order = ['7', '5', '3', '8', '1', '6', '16', '14', '12', '9',
         '11', '13', '15', '2', '10', '4', 'C', 'D', 'B', 'A']
df1 = df1.set_index('Lifeboat').reindex(order).reset_index()

df1.head()

I add the columns `female_per` and `male_per` that correspond to the percentages of females/males (rounded to one decimal value), and the column `Side` that corresponds to the side of the ship from which the lifeboats were launched. Lifeboats with odd numbers were launched from the starboard side and lifeboats with even numbers from the port side.

In [ ]:
df1['female_per'] = df1['female'] / (df1['female'] + df1['male']) * 100
df1['male_per'] = df1['male'] / (df1['female'] + df1['male']) * 100
df1[['female_per', 'male_per']] = df1[['female_per', 'male_per']].round(1)

lifeboat_odd = ['1', '3', '5', '7', '9', '11', '13', '15', 'A', 'C']
df1.loc[df1.Lifeboat.isin(lifeboat_odd), 'Side'] = 'starboard'
df1.loc[~df1.Lifeboat.isin(lifeboat_odd), 'Side'] = 'port'

df1.head()

Below is the code for the bar chart.

In order to interact with data in Bokeh, it is recommended to use its own data object ColumnDataSource (CDS). CDS allows some very nice interactive features like linking, streaming, etc. (see more in the [Bokeh User Guide](https://bokeh.pydata.org/en/latest/docs/user_guide/data.html)). The Pandas DataFrame `df1` can be provided as input to create the CDS `s1`.

The central part of a Bokeh graph is the figure object that can be conveniently created with the function `figure()`. Since the bar chart will be categorical, the list of lifeboats should be passed from the CDS `s1.data['Lifeboat']` to the parameter `x_range`. Note that the list of lifeboats doesn't necessarily have to come from the CDS but I prefer to use only the CDS after it has been initialized. The tools `hover` and `tap`are added to the figure. The parameter `tooltips='@$name'` says that the hover tooltip will show the number of females/males of each bar while hovering over them.

Here I chose to create a stacked vertical bar chart and added the corresponding glyph `vbar_stack` to the figure `p1`. The numbers of females/males in the lifeboats are taken from `s1` that was defined earlier. Some parameters of `p1` (like axis labels, grid lines, legend location, etc.) are adjusted separately.

I would also like to create an HTML content block that will display additional text information about the specific lifeboat when its corresponding bar is selected. For this purpose I can initialize the Div object `div1` that encloses its contents in the HTML tag `<div>`.

The interaction between the figure `p1` and the object `div1` is specified by the method `js_on_event()` that executes `callback1` whenever the user clicks on a bar from the bar chart. `callback1` is a custom JavaScript object that brings additional interactivity. One could also use [Bokeh applications](https://bokeh.pydata.org/en/latest/docs/user_guide/embed.html) that only requires Python but I couldn't figure out how to run Bokeh applications within a Kaggle noteobook.

To test the resulting bar chart, go ahead and click on one of the bars. 😎

In [ ]:
# Create the ColumnDataSource object "s1"
s1 = ColumnDataSource(df1)

# Create the figure object "p1"
p1 = figure(title='Click on a column to display more information',
            plot_width=500, plot_height=325, x_range = s1.data['Lifeboat'],
            toolbar_location=None, tools=['hover', 'tap'], tooltips='@$name')

# Add stacked vertical bars to "p1"
p1.vbar_stack(['female', 'male'], x='Lifeboat', width=0.8, source=s1,
              fill_color=['#66c2a5', '#fc8d62'], line_color=None, legend=['Female', 'Male'])

# Change parameters of "p1"
p1.title.align = 'center'
p1.xaxis.axis_label = 'Lifeboat (in launch order)'
p1.yaxis.axis_label = 'Count'
p1.y_range.start = 0
p1.x_range.range_padding = 0.05
p1.xgrid.grid_line_color = None
p1.legend.orientation = 'horizontal'
p1.legend.location = 'top_left'

# Create the Div object "div1"
div1 = Div()

# Create the custom JavaScript callback
callback1 = CustomJS(args=dict(s1=s1, div1=div1), code='''
    var ind = s1.selected.indices;
    if (String(ind) != '') {
        lifeboat = s1.data['Lifeboat'][ind];
        female = s1.data['female'][ind];
        male = s1.data['male'][ind];
        female_per = s1.data['female_per'][ind];
        male_per = s1.data['male_per'][ind];
        side = s1.data['Side'][ind];
        message = '<b>Lifeboat: ' + String(lifeboat) + ' (' + String(side) + ' side)' + '</b><br>Females: ' + String(female) + ' (' + String(female_per) +  '%)' + '<br>Males: ' + String(male) + ' (' + String(male_per) +  '%)' + '<br>Total: ' + String(female+male);
        div1.text = message;
    }
    else {
        div1.text = '';
    }
''')        

# When tapping the plot "p1" execute the "callback1"
p1.js_on_event('tap', callback1)

# Display "p1" and "div1" as a row
show(row(p1, div1))

It's worth noting that the number of people on this bar chart is lower than the real numbers. This is because there were quite some crew members in the lifeboats who are not included in this passengers list.

Another observation is that most of the lifeboats were not filled to their maximum capacity. The most striking examples is the lifeboat 1 that had only 12 people in it (5 passengers and 7 crew members) with its full capacity of 40 people! The reason behind it was that in the beginning many passengers didn't believe that Titanic would sink.

One might also notice that the number of males in the lifeboats was very high especially in the lifeboats launched from the starboard side (odd numbers). In comparison, the loading crew on the port side was much stricter with the rule "women and children first".

# 3. Interactive pie chart with dropdown menu

The second chart will display distribution of classes among the Titanic survivors depending on the lifeboats. For this purpose I will be using the interactive pie chart and the dropdown menu with the list of all lifeboats. When a lifeboat number is selected in the dropdown menu, the pie chart should update the distribution of classes.

I begin by creating a DataFrame `df2` with the columns `Lifeboat` and `Pclass`. Then I apply the same procedures for cleaning, creating dummy variables, grouping and reordering as in the previous section.

In [ ]:
df2 = full[['Lifeboat', 'Pclass']].copy()

df2 = df2[(df2.Lifeboat.notna()) & (df2.Lifeboat != '?')]

df2.loc[df2.Lifeboat == '14?', 'Lifeboat'] = '14'
df2.loc[df2.Lifeboat == '15?', 'Lifeboat'] = '15'
df2.loc[df2.Lifeboat == 'A[64]', 'Lifeboat'] = 'A'

df2 = pd.get_dummies(df2, columns=['Pclass'], prefix='', prefix_sep='')

df2 = df2.groupby('Lifeboat', as_index=False).sum()

order = ['7', '5', '3', '8', '1', '6', '16', '14', '12', '9',
         '11', '13', '15', '2', '10', '4', 'C', 'D', 'B', 'A']
df2 = df2.set_index('Lifeboat').reindex(order).reset_index()

df2.head()

Instead of the amount of people per class I would prefer to see their percentage per class in a certain lifeboat. Therefore, I add columns `1_per`, `2_per`, `3_per` that indicate percentages of the corresponding classes.

For a pie chart I will need to specify the angles (in radians) for each of the sectors. These angles are added as columns `1_ang`, `2_ang`, `3_ang` and can be easily obtained from the percentages.

In [ ]:
df2['1_per'] = df2['1'] / (df2['1'] + df2['2'] + df2['3']) * 100
df2['2_per'] = df2['2'] / (df2['1'] + df2['2'] + df2['3']) * 100
df2['3_per'] = df2['3'] / (df2['1'] + df2['2'] + df2['3']) * 100


df2['1_ang'] = df2['1_per'] / 100 * 2 * np.pi
df2['2_ang'] = df2['2_per'] / 100 * 2 * np.pi
df2['3_ang'] = df2['3_per'] / 100 * 2 * np.pi

df2.head()

Notice that `df2` contains percentages and angles for all lifeboats. On the pie chart, however, I can only display the distribution of classes for one lifeboat. Therefore, a separate DataFrame `df2_plot` like the one below is needed for plotting. The percentages and angles will be passed from `df2` to `df2_plot` when a particular lifeboat is selected from the dropdown menu.

In [ ]:
df2_plot=pd.DataFrame({'class': ['Class 1', 'Class 2', 'Class 3'],
                       'percent': [float('nan'), float('nan'), float('nan')],
                       'angle': [float('nan'), float('nan'), float('nan')],
                       'color': ['#c9d9d3', '#718dbf', '#e84d60']})
df2_plot

Below is the code for the pie chart.

Just like in the case of the bar chart, first I create the ColumnDataSource (CDS) objects. Here, however, two CDS `s2` and `s2_plot` are needed. `s2` will be used to store the percentages and angles for all lifeboats while `s2_plot` will contain information for only one lifeboat that is plotted at the moment. `s2` will be also used to update `s2_plot` when a different lifeboat is selected from the dropdown menu.

During the initalization of the Figure object `p2` I slightly adjusted the `y_range` so that the legend doesn't cover the pie chart itself. I also changed the parameter `tooltips` to show the percentages rounded to one decimal digit when hovering over the sectors.

The pie chart is created by adding circular sectors using the glyph `wedge()`. Its parameters `start_angle` and `end_angle` are easily calculated from the sector angles using the Bokeh function `cumsum()`.

The dropdown menu is added with the Select object along with the options list. When the user chooses a different value from the dropdown menu, the method `js_on_change()` will execute the `callback2`. It checks if the value in the dropdown menu is not `Please choose...` and then updates `s2_plot` with the information from `s2` for the corresponding lifeboat. Note that `cb_obj` is the built-in Bokeh variable to store the value of the selected option when callbacks are used.

In [ ]:
# Create the ColumnDataSource objects "s2" and "s2_plot"
s2 = ColumnDataSource(df2)
s2_plot = ColumnDataSource(df2_plot)

# Create the Figure object "p2"
p2 = figure(plot_width=275, plot_height=350, y_range=(-0.5, 0.7),
            toolbar_location=None, tools=['hover'], tooltips='@percent{0.0}%')

# Add circular sectors to "p2"
p2.wedge(x=0, y=0, radius=0.8, source=s2_plot,
         start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
         fill_color='color', line_color=None, legend='class')

# Change parameters of "p2"
p2.axis.visible = False
p2.grid.grid_line_color = None
p2.legend.orientation = 'horizontal'
p2.legend.location = 'top_center'

# Create the custom JavaScript callback
callback2 = CustomJS(args=dict(s2=s2, s2_plot=s2_plot), code='''
    var ang = ['1_ang', '2_ang', '3_ang'];
    var per = ['1_per', '2_per', '3_per'];
    if (cb_obj.value != 'Please choose...') {
        var boat = s2.data['Lifeboat'];
        var ind = boat.indexOf(cb_obj.value);
        for (var i = 0; i < ang.length; i++) {
            s2_plot.data['angle'][i] = s2.data[ang[i]][ind];
            s2_plot.data['percent'][i] = s2.data[per[i]][ind];
        }
    }
    else {
        for (var i = 0; i < ang.length; i++) {
            s2_plot.data['angle'][i] = undefined;
            s2_plot.data['percent'][i] = undefined;
        }

    }
    s2_plot.change.emit();
''')

# When changing the value of the dropdown menu execute "callback2"
options = ['Please choose...'] + list(s2.data['Lifeboat'])
select = Select(title='Lifeboat (in launch order)', value=options[0], options=options)
select.js_on_change('value', callback2)

# Display "select" and "p2" as a column
show(column(select, p2))

Notice that first 6 lifeboats were almost exclusively filled with the passengers from the 1st class.

# 4. Choropleth map with reset button

The third chart will display home countries of the Titanic survivors depending on the lifeboats. For this purpose I will be using a choropleth map and will do it in two steps. During the first step, I will import the countries coordinates and plot their boundaries. During the second step, I will fill in the shapes of the obtained countries with the numbers of people from these countries depending on the lifeboats.

## 4.1. Plotting countries boundaries

To import the coordinates of the countries boundaries, one can use the [Natural Earth map datasets](https://github.com/nvkelso/natural-earth-vector). Geopandas can easily import a GeoJSON file and returns a GeoDataFrame. The latter is very similar to a Pandas DataFrame, however, it has additional methods that are useful for handling geospatial data.

In [ ]:
gdf = gpd.read_file('https://raw.githubusercontent.com/nvkelso/natural-earth-vector/master/geojson/ne_110m_admin_0_countries.geojson')
gdf.head()

Let's remove Antarctica from `gdf` since nobody lives there and it occupies a lot of space on the plot (especially in the equirectangular projection). Usingt the Geopandas method `plot()` I quickly check that Antarctica was indeed removed from `gdf`.

In [ ]:
gdf = gdf[gdf.NAME != 'Antarctica']
gdf.plot(figsize=(10, 5));

The coordinates of the countries boundaries are stored in the column `geometry` as Polygon or MultiPolygon objects. When using the method `boundary` these objects return LineString and MultiLineString objects respectively.  The coordinates of a LineString object can be directly accessed using the method `xy`. The coordinates of the MultiLineString object need, however, to be extracted separately for every line. Therefore, the LineString and MultiLineString objects should be treated differently. Note that the boundary coordinates for Bokeh should be 3-times nested lists in order to plot them as multiple multipolygons. This complicated structure allows to distinguish multipolygons from each other, individual polygons inside a multipolygon and holes inside an individual polygon. One should keep this structure in mind when extracting the coordinates and add nested lists where necessary.

Finally, I create a DataFrame `df3_plot` that contains countries names, x-y coordinates of their boundaries and a counter for the number of people. For this test chart I fill the counter with `NaN`.

In [ ]:
xs = []
ys = []
for obj in gdf.geometry.boundary:
    if obj.type == 'LineString':
        obj_x, obj_y = obj.xy
        xs.append([[list(obj_x)]])
        ys.append([[list(obj_y)]])
    elif obj.type == 'MultiLineString':
        obj_x = []
        obj_y = []
        for line in obj:
            line_x, line_y = line.xy
            obj_x.append([list(line_x)])
            obj_y.append([list(line_y)])
        xs.append(obj_x)
        ys.append(obj_y)

country = gdf['NAME'].values        

df3_plot = pd.DataFrame({'country': country, 'xs': xs, 'ys': ys, 'count': float('nan')})

df3_plot.head()

Below is the code for test choropleth map that plots only countries boundaries.

In addition to the `hover` tool, I include `pan` and `wheel_zoom` to easily navigate the map. The hover tooltip is set to display the country name.

For plotting countries boundaries I use the Bokeh glyph `multi_polygons` that takes countries coordinates from `s3_plot`.

To reset the map view, I add a button that executes `callback3_test` when clicking on it. The reset of the chart is also possible using the tool `reset` from the toolbar but I decided not to include the whole toolbar just for one button.

In [ ]:
# Create the ColumnDataSource object "s3_plot"
s3_plot = ColumnDataSource(df3_plot)

# Create the Figure object "p3_test"
p3_test = figure(plot_width=775, plot_height=350,
                 toolbar_location=None, tools=['hover', 'pan', 'wheel_zoom'],
                 active_scroll='wheel_zoom', tooltips='@country')

# Add multipolygons to "p3_test"
p3_test.multi_polygons(xs='xs', ys='ys', fill_color='count', source=s3_plot)

# Change parameters of "p3_test"
p3_test.axis.visible = False
p3_test.grid.grid_line_color = None

# Create the custom JavaScript callback
callback3_test = CustomJS(args=dict(p3_test=p3_test), code='''
    p3_test.reset.emit();
''')    

# When clicking on the button execute "callback3_test"
button = Button(label='Reset view')
button.js_on_click(callback3_test)

# Display "p3_test" and "button" as a column
show(column(p3_test, button))

## 4.2. Filling in countries shapes

To fill in countries shapes I need find the number of people for each country in lifeboats. This information can be extracted from the Titanic extended dataset. I begin by creating the DataFrame `df3` with the columns `Lifeboat` and `Hometown`.

In [ ]:
df3 = full[['Lifeboat', 'Hometown']].copy()
df3.head()

I extract home countries using regular expressions with matched groups. Then I remove the column `Hometown` since it is not needed anymore.

In [ ]:
temp = df3.Hometown.str.extract(r'(?P<Town>.*)\, (?P<Country>.*$)')
df3['Home_country'] = temp['Country']
df3 = df3.drop('Hometown', axis=1, errors='ignore')

df3.head()

Afterwards I apply the same procedure to remove missing values and clean the values as in the previous sections.

In [ ]:
df3 = df3[(df3.Lifeboat.notna()) & (df3.Lifeboat != '?')]

df3.loc[df3.Lifeboat == '14?', 'Lifeboat'] = '14'
df3.loc[df3.Lifeboat == '15?', 'Lifeboat'] = '15'
df3.loc[df3.Lifeboat == 'A[64]', 'Lifeboat'] = 'A'

df3.head()

Note that some countries in `df3` are spelled as abbreviations (for example, "US" or "UK) or have references (for example, "UK[note 3]"), or don't exist anymore (for example, "Russian Empire").

In [ ]:
df3.Home_country.unique()

Therefore, the countries names need to be corrected according to their standard spelling.

In [ ]:
to_replace = [('US', 'United States of America'), ('UK[note 3]', 'India'),
              ('England', 'United Kingdom'), ('UK', 'United Kingdom'),
              ('Channel Islands', 'United Kingdom'), ('Siam', 'Thailand'),
              ('Syria[81]', 'Syria'), ('Scotland', 'United Kingdom'),
              ('British India', 'India'), ('Ireland[note 1]', 'Ireland'),
              ('Russian Empire', 'Russia'), ('Russian Empire[note 6]', 'Finland'),
              ('Siam[note 5]', 'Thailand'), ('German Empire[note 2]', 'Germany'),
              ('British India[note 3]', 'India')]

for old, new in to_replace:
    df3.loc[df3.Home_country == old, 'Home_country'] = new
    
df3.Home_country.unique()

Then I apply the rest of the familiar procedures to group the number of people from different countries for each lifeboat.

In [ ]:
df3 = pd.get_dummies(df3, columns=['Home_country'], prefix='', prefix_sep='')

df3 = df3.groupby('Lifeboat', as_index=False).sum()

order = ['7', '5', '3', '8', '1', '6', '16', '14', '12', '9',
         '11', '13', '15', '2', '10', '4', 'C', 'D', 'B', 'A']
df3 = df3.set_index('Lifeboat').reindex(order).reset_index()

df3.head()

Note, however, that this is not the full list of countries that are displayed on the choropleth map. Therefore, I need to add the columns for all other countries from `d3_plot` and rearrange them in the same order as they appear in `d3_plot`.

In [ ]:
country = df3_plot['country']
diff = country[~country.isin(df3.columns)].values
df3 = pd.concat([df3, pd.DataFrame(columns=diff)], axis=1).fillna(0)
df3 = df3.loc[:, np.append(['Lifeboat'], country.values)]

df3.head()

Below is the code for the complete choropleth map.

In addition to the test choropleth map, I created the colormap `cmap` that is used to fill in multipolygons. The upper limit of `cmap` is set to 1 so that all countries that have more than 1 person are highlighted with dark blue. In this way, it is easier to see smaller countries.

I also adapted the hover tooltip to show the number of people after the country name.

Finally, to choose a lifeboat, I have added the same dropdown menu as in the pie chart.

In [ ]:
# Create the ColumnDataSource objects "s3_plot" and "s3"
s3_plot = ColumnDataSource(df3_plot)
s3 = ColumnDataSource(df3)

# Reverse the palette and create a linear color map
Blues8.reverse()
cmap = linear_cmap('count', palette=Blues8, low=0, high=1)

# Create the Figure object "p3"
p3 = figure(plot_width=775, plot_height=350,
            toolbar_location=None, tools=['hover', 'pan', 'wheel_zoom'],
            active_scroll='wheel_zoom', tooltips='@country: @count')

# Add multipolygons to "p3"
p3.multi_polygons(xs='xs', ys='ys', fill_color=cmap, source=s3_plot)

# Change parameters of "p3"
p3.axis.visible = False
p3.grid.grid_line_color = None

# Create the custom JavaScript callbacks
callback3_select = CustomJS(args=dict(s3=s3, s3_plot=s3_plot), code='''
    var country = s3_plot.data['country'];
    if (cb_obj.value != 'Please choose...') {
        var boat = s3.data['Lifeboat'];
        var ind = boat.indexOf(cb_obj.value);
        for (i = 0; i < country.length; i++) {
            s3_plot.data['count'][i] = s3.data[country[i]][ind];
        }
    }
    else {
        for (i = 0; i < country.length; i++) {
            s3_plot.data['count'][i] = undefined;
        }
    }
    s3_plot.change.emit();
''')

callback3_button = CustomJS(args=dict(p3=p3), code='''
    p3.reset.emit();
''')
    
# When changing the value of the dropdown menu execute "callback3_select"
options = ['Please choose...'] + list(s3.data['Lifeboat'])
select = Select(title='Lifeboat (in launch order)', value=options[0], options=options)
select.js_on_change('value', callback3_select)

# When clicking on the reset button execute "callback3_button"
button = Button(label='Reset view')
button.js_on_click(callback3_button)

# Display "select", "p3", and "button" as a column
show(column(select, p3, button))

# 5. Interactive dashboard of all three charts

Now that we have all 3 charts, let's combine them in one dashboard. I don't have to initialize the charts again but can simply display them in a preferred layout.

The only new element here is the `callback4` that updates the other two charts when a specific bar from the bar chart is tapped.

In [ ]:
# Create the custom JavaScript callback
callback4 = CustomJS(args=dict(s1=s1, s2=s2, s3=s3, s2_plot=s2_plot, s3_plot=s3_plot), code='''
    var ind = s1.selected.indices;
    var ang = ['1_ang', '2_ang', '3_ang'];
    var per = ['1_per', '2_per', '3_per'];
    var country = s3_plot.data['country'];
    if (String(ind) != '') {
        for (i = 0; i < ang.length; i++) {
            s2_plot.data['angle'][i] = s2.data[ang[i]][ind];
            s2_plot.data['percent'][i] = s2.data[per[i]][ind];
        }
        for (i = 0; i < country.length; i++) {
            s3_plot.data['count'][i] = s3.data[country[i]][ind];
        }
    }
    else {
        for (i = 0; i < ang.length; i++) {
            s2_plot.data['angle'][i] = undefined;
            s2_plot.data['percent'][i] = undefined;
        }
        for (i = 0; i < country.length; i++) {
            s3_plot.data['count'][i] = undefined;
        }
    }
    s2_plot.change.emit();
    s3_plot.change.emit();
''')    
    
# When tapping the plot "p1" execute "callback4"
p1.js_on_event('tap', callback4)

# Display "p1","p2", "p3" and "button" in the specified layout
l = layout([[p1, p2], [p3], [button]])
show(l)

Thanks for **UPVOTING** this kernel! Trying to become a Kernels Master. 🤘

Check out my other cool projects:
- [💲 Minimizing investment risk for high interest loans](https://www.kaggle.com/pavlofesenko/minimizing-investment-risk-for-high-interest-loans)
- [🌐 Extending Titanic dataset using Wikipedia](https://www.kaggle.com/pavlofesenko/extending-titanic-dataset-using-wikipedia)
- [👪 Titanic extended dataset (Kaggle + Wikipedia)](https://www.kaggle.com/pavlofesenko/titanic-extended)